# Programación declarativa @ URJC
# Programación funcional
## Curso 21-22, convocatoria ordinaria (27 de octubre de 2021)
## Campus de Vicálvaro


# Definiciones auxiliares

In [55]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

### Algunas definiciones de tipos y funciones auxiliares

In [19]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

object Tree{
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)
}

import Tree._

def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    tree match {
        case Empty() => empty
        case Node(left, root, right) => node(foldTree(left)(empty)(node), root, foldTree(right)(empty)(node))
    }

defined class Tree
defined class Empty
defined class Node
defined object Tree
import Tree._


defined function foldTree

In [78]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
 
        // Returns the number of elements of this list
        def length: Int
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard

In [80]:
type Not[P] = P => Nothing

defined type Not

# Ejercicio 1


__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la siguiente tautología de la lógica proposicional intuicionista: 

$\neg p \rightarrow \neg\neg\neg p$

In [65]:
def proof[P]: Not[P] => Not[Not[Not[P]]] = 
    (notP: P => Nothing) => 
        (f: (P => Nothing) => Nothing) => 
            f(notP) : Nothing

defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$(\neg q \rightarrow \neg p) \rightarrow (p \rightarrow q)$

Supóngase para ello que la ley del tercio excluso se cumple para la variable proposicional $q$, es decir, que la fórmula $q \vee \neg q$  puede utilizarse como premisa.


In [62]:
def proof[P, Q](middle: Either[Q, Not[Q]]): (Not[Q] => Not[P]) => (P => Q) = 
    (f: Not[Q] => Not[P]) => (p: P) => 
        middle match {
            case Left(q: Q) => 
                q : Q
            case Right(notQ: Not[Q]) => 
                f(notQ)(p) : Q
        }

defined function proof

# Ejercicio 2
__(1 punto)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$(1+1)^X \cong Boolean^X$

de tal forma que se verifique el siguiente test unitario para $X=Int$:

In [107]:
class IsoTest(
    from: (Int => Either[Unit, Unit]) => Int => Boolean, 
    to: (Int => Boolean) => Int => Either[Unit, Unit]
) extends FlatSpec with Matchers{
    
    val f: Int => Either[Unit, Unit] = 
        i => if (i % 2 == 0) Left(()) else Right(())
    
    val g: Int => Boolean = 
        _ % 2 == 0
    
    "from-to" should "work" in {
        from(to(g))(0) shouldBe g(0)
        from(to(g))(1) shouldBe g(1)
        from(to(g))(2) shouldBe g(2)
        from(to(g))(3) shouldBe g(3)
    }
    
    "to-from" should "work" in {
        to(from(f))(0) shouldBe f(0)
        to(from(f))(1) shouldBe f(1)
        to(from(f))(2) shouldBe f(2)
        to(from(f))(3) shouldBe f(3)
    }
}

defined class IsoTest

In [104]:
def from[X](l: X => Either[Unit, Unit]): X => Boolean = 
    x => l(x) match {
        case Left(()) => true
        case Right(()) => false
    }

defined function from

In [105]:
def to[X](l: X => Boolean): X => Either[Unit, Unit] = 
    x => if (l(x)) Left(()) else Right(())

defined function to

In [106]:
run(new IsoTest(from[Int], to[Int]))

cmd102$Helper$IsoTest:
from-to
- should work
to-from
- should work


# Ejercicio 3
__(3 puntos)__

La función `slice` recibe una lista de valores de tipo `X` y un rango de posiciones, y devuelve una lista con los elementos comprendidos dentro de ese rango. El comportamiento de la función se ilustra en el siguiente test unitario, donde la función `slice` se encuentra particularizada para el tipo `X=Int`:


In [56]:
class TestSlice(
    slice: List[Int] => (Int, Int) => List[Int]
) extends FlatSpec with Matchers{
    "slice" should "work" in {
        slice(List())(0,3) shouldBe List()
        slice(List(1,2,3,4))(5,6) shouldBe List()
        slice(List(1,2,3,4))(0,2) shouldBe List(1,2,3)
        slice(List(1,2,3,4))(0,6) shouldBe List(1,2,3,4)
        slice(List(1,2,3,4))(1,3) shouldBe List(2,3,4)
        slice(List(1,2,3,4))(1,2) shouldBe List(2,3)
    }
}

defined class TestSlice

__a) (1 punto)__ Implementa la función `slice` con recursión final (o recursión por la cola).

In [57]:
def slice[A](list: List[A])(from: Int, to: Int): List[A] = {
    def auxSlice(aux: List[A])(out: List[A], idx: Int): List[A] = 
        aux match {
            case e :: tail if from <= idx && idx <= to => 
                auxSlice(tail)(e :: out, idx + 1)
            case _ :: tail if idx < from => 
                auxSlice(tail)(out, idx + 1)
            case _ => 
                out
        }
    
    auxSlice(list)(Nil, 0).reverse
}

defined function slice

In [58]:
run(new TestSlice(slice))

cmd55$Helper$TestSlice:
slice
- should work


__b) (1 punto)__ Implementa la función `slice` con `foldLeft`.

In [59]:
def slice[A](list: List[A])(from: Int, to: Int): List[A] = 
    list.foldLeft((Nil: List[A], 0)){
        case ((out, idx), e) if from <= idx && idx <= to => 
            (e :: out, idx + 1)
        case ((out, idx), _) => 
            (out, idx + 1)
    }._1.reverse

defined function slice

In [60]:
run(new TestSlice(slice))

cmd55$Helper$TestSlice:
slice
- should work


# Ejercicio 4
__(3 puntos)__

Considérese una función que dado un árbol binario devuelve el camino más largo desde la raíz a cualquier de sus hojas. Si existen varios caminos con la misma longitud máxima, la función deberá devolver uno cualquiera de ellos.

In [67]:
class TestLongestPath(longest: Tree[Int] => List[Int]) extends FlatSpec with Matchers{
    "longest path" should "work" in {
        longest(void) shouldBe 
            List()
        
        longest(left(left(right(3,right(2,leaf(1))), 4), 5)) shouldBe 
            List(5,4,3,2,1)
        
        longest(node(left(leaf(4), 1), 0, 
                     node(leaf(3), 2, right(2, right(4, leaf(5)))))) shouldBe 
            List(0, 2, 2, 4, 5)
        
        longest(node(left(right(0, leaf(1)), 2), 3, node(left(leaf(5), 4), 9, leaf(7)))) should 
            (equal(List(3, 2, 0, 1)) or equal(List(3, 9, 4, 5)))
    }
}

defined class TestLongestPath

__a) (1,5 puntos)__ Implementa la función recursivamente. La implementación podrá hacer uso del método `length` de la clase `List[A]`.

In [70]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    tree match {
        case Empty() => List()
        case Node(left, root, right) => 
            val longestLeft: List[A] = longestPath(left)
            val longestRight: List[A] = longestPath(right)
            root :: (if (longestRight.length > longestLeft.length) longestRight
                     else longestLeft)
    }

defined function longestPath

In [71]:
run(new TestLongestPath(longestPath))

cmd66$Helper$TestLongestPath:
longest path
- should work


__b) (1,5 puntos)__ Implementa la función mediante la función de orden superior `foldTree`. La implementación podrá hacer uso del método `length` de la clase `List[A]`.

In [72]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    foldTree(tree)(List[A]())(
        (longestLeft, root, longestRight) => 
            root :: (if (longestLeft.length > longestRight.length) longestLeft 
                     else longestRight)
    )

defined function longestPath

In [75]:
run(new TestLongestPath(longestPath))

cmd66$Helper$TestLongestPath:
longest path
- should work


__c) (1,5 puntos)__ Implementa la función mediante la función de orden superior `foldTree`, __sin__ hacer uso de la función `length`.

In [76]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    foldTree(tree)((0, List[A]())){
        case ((l, longestLeft), root, (r, longestRight)) => 
            if (l > r) (l+1, root :: longestLeft) 
            else (r+1, root :: longestRight)
    }._2

defined function longestPath

In [77]:
run(new TestLongestPath(longestPath))

cmd66$Helper$TestLongestPath:
longest path
- should work


# Ejercicio 5
__(3 puntos)__

Hallar el camino más largo entre la raíz y las hojas de un árbol binario. 

In [81]:
def dyvG[P, B, C, S](problem: P)(
                 decompose: P => Either[B, (C, P, P)],
                 solve: B => S,
                 compose: (C, S, S) => S): S = 
    decompose(problem) match {
        case Left(base) => solve(base)
        case Right((aux, problem1, problem2)) => 
            compose(aux, dyvG(problem1)(decompose, solve, compose),
                              dyvG(problem2)(decompose, solve, compose))
    }

defined function dyvG

In [43]:
def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    dyv[Tree[A], Empty[A], A, B](tree)(
        { case Empty() => Left(Empty()); 
          case Node(left, root, right) => Right((root, left, right)) },
        { _ => empty },
        (root: A, sol1: B, sol2: B) => node(sol1, root, sol2))

defined function foldTree

In [82]:
def dyv[P, S](problem: P)(
              decompose: P => Either[P, (P, P)],
              solve: P => S,
              compose: (S, S) => S): S = 
    decompose(problem) match {
        case Left(base) => solve(base)
        case Right((problem1, problem2)) => 
            compose(dyv(problem1)(decompose, solve, compose),
                    dyv(problem2)(decompose, solve, compose))
    }

defined function dyv

In [83]:
def split(numbers: Array[Int]): Either[Array[Int], (Array[Int], Array[Int])] = 
    if (numbers.length <= 1) Left(numbers)
    else Right(numbers.slice(0, numbers.length/2), numbers.slice(numbers.length/2, numbers.length))

defined function split

In [85]:
def solve(numbers: Array[Int]): Array[Int] = 
    numbers

defined function solve

In [86]:
def merge(array1: Array[Int], array2: Array[Int]): Array[Int] = 
    (array1, array2) match {
      case (Array(), Array()) => Array.empty
      case (Array(), ys2)         => ys2
      case (xs2, Array())         => xs2
      case (xs1@Array(x, tail1@_*), ys1@Array(y, tail2@ _*)) =>
        if (x < y) x +: merge(tail1.toArray, ys1)
        else y +: merge(xs1, tail2.toArray)
    }

defined function merge

In [87]:
def mergeSort(numbers: Array[Int]): Array[Int] = 
    if (numbers.length <= 1) numbers
    else merge(mergeSort(numbers.slice(0, numbers.length/2)), 
               mergeSort(numbers.slice(numbers.length/2, numbers.length)))

defined function mergeSort

In [91]:
def mergeSort(numbers: Array[Int]): Array[Int] = 
    dyv(numbers)(split, a => a, merge)

defined function mergeSort

In [92]:
mergeSort(Array(8,7,6,5,4,3,2,1))

res91: Array[Int] = Array(1, 2, 3, 4, 5, 6, 7, 8)

In [54]:
dyv(Array(8,7,6,5,4,3,2,1))(
    split, 
    solve, 
    (_ : Unit, a1: Array[Int], a2: Array[Int]) => merge(a1, a2))

res53: Array[Int] = Array(1, 2, 3, 4, 5, 6, 7, 8)